
**Objective:**
The objective of this challenge is to build a machine learning model that helps predict the location and extent of floods in southern Malawi.


**Evaluation:**
RMSE

**Submission:**
For every row in the dataset, submission files should contain 2 columns: Square_ID and target_2019.



In [9]:
#load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder  
from sklearn.preprocessing import StandardScaler  
from scipy import sparse
from catboost import CatBoostRegressor
import lightgbm as lgb
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
import random
import dython
from dython import nominal
from catboost import CatBoostClassifier, Pool
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression



In [16]:
#load data
df = pd.read_csv("Train.csv")
submission = pd.read_csv("SampleSubmission.csv")

## Data Analysis

In [17]:
df.head()

,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode,Square_ID
0,34.26,-15.91,0.0,887.764222,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3896-14ce-11ea-bce5-f49634744a41
1,34.26,-15.90,0.0,743.403912,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3897-14ce-11ea-bce5-f49634744a41
2,34.26,-15.89,0.0,565.728343,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,9,4e3c3898-14ce-11ea-bce5-f49634744a41
3,34.26,-15.88,0.0,443.392774,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c3899-14ce-11ea-bce5-f49634744a41
4,34.26,-15.87,0.0,437.443428,0.0,0.0,0.0,14.844025,14.552823,12.237766,...,0.896323,1.68,0.0,0.0,0.0,0.0,0.0,0.0,10,4e3c389a-14ce-11ea-bce5-f49634744a41


In [18]:
df.tail()

,X,Y,target_2015,elevation,precip 2014-11-16 - 2014-11-23,precip 2014-11-23 - 2014-11-30,precip 2014-11-30 - 2014-12-07,precip 2014-12-07 - 2014-12-14,precip 2014-12-14 - 2014-12-21,precip 2014-12-21 - 2014-12-28,...,precip 2019-03-24 - 2019-03-31,precip 2019-03-31 - 2019-04-07,precip 2019-04-07 - 2019-04-14,precip 2019-04-14 - 2019-04-21,precip 2019-04-21 - 2019-04-28,precip 2019-04-28 - 2019-05-05,precip 2019-05-05 - 2019-05-12,precip 2019-05-12 - 2019-05-19,LC_Type1_mode,Square_ID
16461,35.86,-15.44,0.0,635.675022,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072,10,4e6f5dfd-14ce-11ea-bce5-f49634744a41
16462,35.86,-15.43,0.0,632.598892,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072,10,4e6f5dfe-14ce-11ea-bce5-f49634744a41
16463,35.86,-15.42,0.0,632.450136,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072,10,4e6f5dff-14ce-11ea-bce5-f49634744a41
16464,35.86,-15.41,0.0,629.272733,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072,10,4e6f5e00-14ce-11ea-bce5-f49634744a41
16465,35.86,-15.40,0.0,626.164641,16.956563,31.155531,12.882013,8.810145,6.179829,9.863685,...,11.335339,30.984762,0.518269,5.770066,14.839779,4.928294,10.526186,18.746072,10,4e6f5e01-14ce-11ea-bce5-f49634744a41


In [19]:
# Previewing some statistical summaries of the dataframe
# Transposing for a better view
#
df.describe().T

,count,mean,std,min,25%,50%,75%,max
X,16466.0,35.077656,0.392395,34.260000,34.760000,35.050000,35.390000,35.860000
Y,16466.0,-15.813802,0.359789,-16.640000,-16.070000,-15.800000,-15.520000,-15.210000
target_2015,16466.0,0.076609,0.228734,0.000000,0.000000,0.000000,0.000000,1.000000
elevation,16466.0,592.848206,354.790357,45.541444,329.063852,623.000000,751.434813,2803.303645
precip 2014-11-16 - 2014-11-23,16466.0,1.610760,4.225461,0.000000,0.000000,0.000000,1.261848,19.354969
precip 2014-11-23 - 2014-11-30,16466.0,2.502058,8.631846,0.000000,0.000000,0.000000,0.000000,41.023858
precip 2014-11-30 - 2014-12-07,16466.0,1.162076,4.396676,0.000000,0.000000,0.000000,0.000000,22.020803
precip 2014-12-07 - 2014-12-14,16466.0,8.270610,4.263375,1.411452,5.548440,7.941822,10.887235,18.870675
precip 2014-12-14 - 2014-12-21,16466.0,8.892459,3.760052,3.580342,5.905440,8.618390,10.960668,23.044340
precip 2014-12-21 - 2014-12-28,16466.0,9.572821,4.523767,1.254098,6.179885,8.786780,12.670775,21.757828


In [21]:
# Checking for the shape and size of the dataframe
#
print ("The shape is " + str(df.shape) + " and the size is " + str(df.size))

The shape is (16466, 40) and the size is 658640


In [24]:
# Checking for missing values
#
if (df.isnull().sum().any()):
    print("There is missing data")
else:
    print("There is no missing data")

There is no missing data


In [25]:
# Checking if the columns are represented with the appriopriate datatypes
#
df.dtypes

X                                 float64
Y                                 float64
target_2015                       float64
elevation                         float64
precip 2014-11-16 - 2014-11-23    float64
precip 2014-11-23 - 2014-11-30    float64
precip 2014-11-30 - 2014-12-07    float64
precip 2014-12-07 - 2014-12-14    float64
precip 2014-12-14 - 2014-12-21    float64
precip 2014-12-21 - 2014-12-28    float64
precip 2014-12-28 - 2015-01-04    float64
precip 2015-01-04 - 2015-01-11    float64
precip 2015-01-11 - 2015-01-18    float64
precip 2015-01-18 - 2015-01-25    float64
precip 2015-01-25 - 2015-02-01    float64
precip 2015-02-01 - 2015-02-08    float64
precip 2015-02-08 - 2015-02-15    float64
precip 2015-02-15 - 2015-02-22    float64
precip 2015-02-22 - 2015-03-01    float64
precip 2015-03-01 - 2015-03-08    float64
precip 2015-03-08 - 2015-03-15    float64
precip 2019-01-20 - 2019-01-27    float64
precip 2019-01-27 - 2019-02-03    float64
precip 2019-02-03 - 2019-02-10    

## Data modeling

In [26]:
# Creating lists of columns to be used in separating the dataframe into training and testing datasets
#
precip_features_2019 = []
precip_features_2015 = []
for col in df.columns:
  if '2019' in col:
    precip_features_2019.append(col)
  elif 'precip 2014' in col:
    precip_features_2015.append(col)
  elif 'precip 2015' in col:
    precip_features_2015.append(col)

In [27]:
# Separating the train dataset from the main dataframe
#
train = df[df.columns.difference(precip_features_2019)]


In [28]:
# Separating the test dataset from the main dataframe
precip_features_2019.extend(['X','Y','elevation', 'LC_Type1_mode','Square_ID'])
test = df[precip_features_2019]

In [ ]:
#rename columns
new_2015_cols = {}
for col, number in zip(precip_features_2015, range(1, len(precip_features_2015) + 1)):
  if 'precip' in col:
    new_2015_cols[col] = 'week_' + str(number) + '_precip'

    
# Creating a dictionary of column names to be renamed for the testing dataset
#
new_2019_cols = {}
for col, number in zip(precip_features_2019, range(1, len(precip_features_2019) + 1)):
  if 'precip' in col:
    new_2019_cols[col] = 'week_' + str(number) + '_precip'
    
# Renaming the columns
#
train.rename(columns = new_2015_cols, inplace = True)
test.rename(columns = new_2019_cols, inplace = True)

In [29]:
#realign train and test
# Separating the target variable
#
target = train.target_2015


# Aligning the training and testing datasets
#
train, test = train.align(test, join = 'inner', axis = 1)


# Previewing the first three rows of the cleaned and realigned test set
#
test.head(3)

,LC_Type1_mode,Square_ID,X,Y,elevation
0,9,4e3c3896-14ce-11ea-bce5-f49634744a41,34.26,-15.91,887.764222
1,9,4e3c3897-14ce-11ea-bce5-f49634744a41,34.26,-15.90,743.403912
2,9,4e3c3898-14ce-11ea-bce5-f49634744a41,34.26,-15.89,565.728343


## Model selection

In [31]:
#set.seed(1)
# Creating a list of regressor algorithms to compare with
#
models = [RandomForestRegressor(), GradientBoostingRegressor(), AdaBoostRegressor(), DecisionTreeRegressor(),  XGBRegressor(objective ='reg:squarederror'),\
          SVR(), KNeighborsRegressor(), LinearRegression(), CatBoostRegressor(logging_level='Silent')]


# Creating lists of the algorithms, to store the accuracy scores of each fold
#
RandomForest, GradientBoosting, AdaBoost, DecisionTree, XGB, SVR, KNeighbors, Linear, Cat = ([] for x in range(9))


# Creating a list containig the list of each algorithm. Created for easy iteration
#
model_list = [RandomForest, GradientBoosting, AdaBoost, DecisionTree, XGB, SVR, KNeighbors, Linear, Cat]


# Spliting the data into features and the target variable
#
X = train.drop('Square_ID', axis = 1)
y = target


# Creating a cross validation of 10 folds
#
kfold  = KFold(n_splits=10, random_state=101)


# Iterating through each model and appending the scores of each fold to the appriopriate list
#
for i, j in zip(models, model_list):
  j.extend(list(cross_val_score(i, X, y, scoring = 'neg_mean_squared_error', cv = kfold)))

  
# Creating a function to convert neg_mean_squared_error to a square root
#
def sq(lis):
  new_lis = []
  lis = np.array(lis)
  for i in lis:
    i = np.sqrt(i*-1)
    new_lis.append(i)
  return new_lis


# Creating a dataframe of all the rmses from the iterations for each model
#
rmses = pd.DataFrame({'Fold': np.arange(1, 11), 'RandomForest': sq(RandomForest), 'GradientBoosting': sq(GradientBoosting), 'Adaboost': sq(AdaBoost), 'DecisionTree': sq(DecisionTree),\
                       'XGB': sq(XGB), 'SVR': sq(SVR), 'Kneighbors': sq(KNeighbors), 'Linear': sq(Linear), 'Cat': sq(Cat)})

# Setting the index
#
rmses.set_index('Fold', inplace = True)


# Calculating the mean and standard deviation rmse of each algorithm
#
rmses.loc['mean'] = rmses.mean()
rmses.loc['std'] = rmses.std()


# Previewing the rmses dataframe
#
rmses

,RandomForest,GradientBoosting,Adaboost,DecisionTree,XGB,SVR,Kneighbors,Linear,Cat
Fold,,,,,,,,,
1,0.086517,0.085979,0.085575,0.086571,0.086480,0.118377,0.086987,0.106142,0.085531
2,0.070112,0.063486,0.070683,0.092865,0.063111,0.100789,0.070120,0.101283,0.073640
3,0.109590,0.088016,0.104493,0.121539,0.086771,0.116366,0.094812,0.127625,0.106748
4,0.167350,0.169104,0.213211,0.219040,0.165420,0.202579,0.212893,0.254629,0.163917
5,0.164711,0.177434,0.236461,0.199510,0.175604,0.201358,0.197473,0.337343,0.158580
6,0.116066,0.120890,0.146531,0.134537,0.119991,0.167759,0.146516,0.243506,0.109552
7,0.056983,0.065408,0.108939,0.065703,0.063373,0.113365,0.087057,0.126710,0.056078
8,0.150066,0.109135,0.171345,0.140774,0.110136,0.133698,0.186108,0.135270,0.153153
9,0.230737,0.263281,0.250353,0.270462,0.262547,0.266461,0.260355,0.326016,0.245334


In [32]:
# find the best model
#
rmses.loc['mean'].idxmin(), rmses.loc['mean'].min()

('XGB', 0.13615721939813494)

In [33]:
# Arranging the models in ascending order
#
rmses.loc['mean'].sort_values()

XGB                 0.136157
GradientBoosting    0.136657
Cat                 0.137829
RandomForest        0.140812
Kneighbors          0.155204
SVR                 0.164365
DecisionTree        0.164573
Adaboost            0.169851
Linear              0.204403
Name: mean, dtype: float64

In [34]:
# Creating a list of models to use
models = [XGBRegressor(objective ='reg:squarederror'), CatBoostRegressor(logging_level='Silent'), GradientBoostingRegressor()]
model_names = ['xgboost', 'catboost', 'gradientboost']


# Selecting the training features and the target feature
#
X = train.drop('Square_ID', axis = 1)
y = target


# Submission dataset
#
sub = test.drop('Square_ID', axis = 1)


# Using a for loop to create a submission file for each model
#
for model, model_name in zip(models, model_names):
  regressor = model                      # instantiating the model
  regressor.fit(X, y)                    # Training the model
  predictions  = regressor.predict(sub)  # Making predictions
  submission_df = pd.DataFrame({'Square_ID': test.Square_ID, 'target_2019': predictions}) # Creating a submission file
  submission_df.to_csv(model_name + '_baseline.csv', index = False)